In [1]:
import normals_lib
import pointareas as pareas
import kaolin as kal
import torch
import numpy as np
import meshplot as mp

mesh = kal.rep.TriangleMesh.from_obj("horse.obj")

In [3]:
normals = normals_lib.compute_simple_vertex_normals(mesh)
pointareas, cornerareas = pareas.compute_pointareas(mesh)
normals_lib.visualize_normals(mesh, normals.cpu())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.991888…

In [4]:
# Calculate principal curvatures and curvature directions
import curvature
import importlib

k1, k2, e1, e2 = curvature.compute_curvatures(mesh, method='cholesky', normals=normals, pointareas=pointareas,
                                              cornerareas=cornerareas)

In [ ]:
altk1, altk2, _, _ = curvature.compute_curvatures(mesh, method="cholesky", normals=normals,
                                                  pointareas=pointareas, cornerareas=cornerareas)

In [5]:
import dcurv

dc = dcurv.compute_dcurvs(mesh, method='cholesky', curvs=(k1,k2,e1,e2), normals=normals, pointareas=pointareas,
                          cornerareas=cornerareas)

In [21]:
importlib.reload(dcurv)

<module 'dcurv' from '/home/danniccs/rtsc_to_torch/dcurv.py'>

In [91]:
DwKr = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
kr = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
for i in range(0, 12):
    DwKr[i], kr[i] = dcurv.compute_DwKr(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=dc, view_coords=viewpos)

In [92]:
for i in range(0, 12):
    kupper = int(DwKr[i].shape[0] * 0.95)
    klower = int(DwKr[i].shape[0] * 0.05)
    dwupper, _ = torch.kthvalue(DwKr[i], kupper)
    dwlower, _ = torch.kthvalue(DwKr[i], klower)
    DwKr[i][DwKr[i] > dwupper] = dwupper
    DwKr[i][DwKr[i] < dwlower] = dwlower
    kupper = int(kr[i].shape[0] * 0.95)
    klower = int(kr[i].shape[0] * 0.05)
    krupper, _ = torch.kthvalue(kr[i], kupper)
    krlower, _ = torch.kthvalue(kr[i], klower)
    kr[i][kr[i] > krupper] = krupper
    kr[i][kr[i] < krlower] = krlower

In [6]:
kupper = int(k1.shape[0] * 0.95)
klower = int(k1.shape[0] * 0.05)
dwupper, _ = torch.kthvalue(k1, kupper)
dwlower, _ = torch.kthvalue(k1, klower)
k1[k1 > dwupper] = dwupper
k1[k1 < dwlower] = dwlower
kupper = int(k2.shape[0] * 0.95)
klower = int(k2.shape[0] * 0.05)
krupper, _ = torch.kthvalue(k2, kupper)
krlower, _ = torch.kthvalue(k2, klower)
k2[k2 > krupper] = krupper
k2[k2 < krlower] = krlower

In [22]:
import shared_algs
ndotv = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
krf = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
sctest_num = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
sctest_den = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
sctest = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))

tridc = tridc.cuda()
for i in range(0, 12):
    ndotv[i], krf[i], _, sctest_num[i], sctest_den[i] = shared_algs.compute_perview(mesh, normals=normals,
                                                                                    curvs=(trik1,trik2,trie1,trie2),
                                                                                    dcurv=tridc,
                                                                                    view_coords=viewpos)
    sctest[i] = sctest_num[i] / sctest_den[i]

In [23]:
for i in range(0, 12):
    #sctest_den[i][sctest_den[i] == 0] = 0.01
    kupper = int(sctest[i].shape[0] * 0.95)
    klower = int(sctest[i].shape[0] * 0.05)
    scupper, _ = torch.kthvalue(sctest[i], kupper)
    sclower, _ = torch.kthvalue(sctest[i], klower)
    sctest[i][sctest[i] > scupper] = scupper
    sctest[i][sctest[i] < sclower] = sclower
    
    kupper = int(krf[i].shape[0] * 0.95)
    klower = int(krf[i].shape[0] * 0.05)
    scupper, _ = torch.kthvalue(krf[i], kupper)
    sclower, _ = torch.kthvalue(krf[i], klower)
    krf[i][krf[i] > scupper] = scupper
    krf[i][krf[i] < sclower] = sclower

In [86]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[0].to(device=torch.device('cpu')).numpy(),
        shading={'colormap': "jet_r"})